In [0]:
import numpy as np
from scipy.special import binom
import matplotlib.pyplot as plt
import cv2
import IPython
import PIL
import random
from skimage import io
import numpy as np


bernstein = lambda n, k, t: binom(n,k)* t**k * (1.-t)**(n-k)

def bezier(points, num=200):
    N = len(points)
    t = np.linspace(0, 1, num=num)
    curve = np.zeros((num, 2))
    for i in range(N):
        curve += np.outer(bernstein(N - 1, i, t), points[i])
    return curve

class Segment():
    def __init__(self, p1, p2, angle1, angle2, **kw):
        self.p1 = p1; self.p2 = p2
        self.angle1 = angle1; self.angle2 = angle2
        self.numpoints = kw.get("numpoints", 100)
        r = kw.get("r", 0.3)
        d = np.sqrt(np.sum((self.p2-self.p1)**2))
        self.r = r*d
        self.p = np.zeros((4,2))
        self.p[0,:] = self.p1[:]
        self.p[3,:] = self.p2[:]
        self.calc_intermediate_points(self.r)

    def calc_intermediate_points(self,r):
        self.p[1,:] = self.p1 + np.array([self.r*np.cos(self.angle1),
                                    self.r*np.sin(self.angle1)])
        self.p[2,:] = self.p2 + np.array([self.r*np.cos(self.angle2+np.pi),
                                    self.r*np.sin(self.angle2+np.pi)])
        self.curve = bezier(self.p,self.numpoints)


def get_curve(points, **kw):
    segments = []
    for i in range(len(points)-1):
        seg = Segment(points[i,:2], points[i+1,:2], points[i,2],points[i+1,2],**kw)
        segments.append(seg)
    curve = np.concatenate([s.curve for s in segments])
    return segments, curve

def ccw_sort(p):
    d = p-np.mean(p,axis=0)
    s = np.arctan2(d[:,0], d[:,1])
    return p[np.argsort(s),:]

def get_bezier_curve(a, rad=0.2, edgy=0):
    """ given an array of points *a*, create a curve through
    those points. 
    *rad* is a number between 0 and 1 to steer the distance of
          control points.
    *edgy* is a parameter which controls how "edgy" the curve is,
           edgy=0 is smoothest."""
    p = np.arctan(edgy)/np.pi+.5
    a = ccw_sort(a)
    a = np.append(a, np.atleast_2d(a[0,:]), axis=0)
    d = np.diff(a, axis=0)
    ang = np.arctan2(d[:,1],d[:,0])
    f = lambda ang : (ang>=0)*ang + (ang<0)*(ang+2*np.pi)
    ang = f(ang)
    ang1 = ang
    ang2 = np.roll(ang,1)
    ang = p*ang1 + (1-p)*ang2 + (np.abs(ang2-ang1) > np.pi )*np.pi
    ang = np.append(ang, [ang[0]])
    a = np.append(a, np.atleast_2d(ang).T, axis=1)
    s, c = get_curve(a, r=rad, method="var")
    x,y = c.T
    return x,y, a


def get_random_points(n=5, scale=0.8, mindst=None, rec=0):
    """ create n random points in the unit square, which are *mindst*
    apart, then scale them."""
    mindst = mindst or .7/n
    a = np.random.rand(n,2)
    d = np.sqrt(np.sum(np.diff(ccw_sort(a), axis=0), axis=1)**2)
    if np.all(d >= mindst) or rec>=200:
        return a*scale
    else:
        return get_random_points(n=n, scale=scale, mindst=mindst, rec=rec+1)

In [0]:
def generate_blob(im_path, xmax=640, ymax=640, xmin=40, ymin=40):
    fig, ax = plt.subplots()
    ax.set_aspect("equal")

    rad = 0.15
    edgy = 0.1
    n_points = random.randint(4, 8)

    cx = random.randint(xmin, xmax-xmin)
    cy = random.randint(ymin, ymax-ymin)

    scale = random.randint(int(xmax*0.08), int(xmax*0.3))
    opacity = random.randint(30, 80) / 100

    a = get_random_points(n=n_points, scale=scale)
    x,y, _ = get_bezier_curve(a,rad=rad, edgy=edgy)
    ax.fill(x, y, alpha=opacity)
    # plt.show()
    plt.axis('off')
    plt.savefig(im_path, transparent=True)

In [0]:

def logoOverlay(image,logo,alpha=1.0,x=0, y=0, scale=1.0):
    (h, w) = image.shape[:2]
    image = np.dstack([image, np.ones((h, w), dtype="uint8") * 255])
    overlay = cv2.resize(logo, None,fx=scale,fy=scale)
    (wH, wW) = overlay.shape[:2]
    output = image.copy()
    # blend the two images together using transparent overlays
    try:
        if x<0 : x = w+x
        if y<0 : y = h+y
        if x+wW > w: wW = w-x  
        if y+wH > h: wH = h-y
        print(x,y,wW,wH)
        opacity= random.randint(1, 4) / 10
        overlay=cv2.addWeighted(output[y:y+wH, x:x+wW],alpha,overlay[:wH,:wW],opacity,0)
        output[y:y+wH, x:x+wW ] = overlay
    except Exception as e:
        print("Error: Logo position is overshooting image!")
        print(e)

    output= output[:,:,:3]
    return output

In [0]:
def overlay_blobs(im_path):
  background = cv2.imread(im_path)
  xmax, ymax = background.shape[:2]
  n_blobs = random.randint(2, 3)
  for b in range(n_blobs):
    blob_id = random.randint(0, 400)
    blob_path = f"img/{blob_id}.png"
    overlay = cv2.imread(blob_path, cv2.IMREAD_UNCHANGED)
    overlay[np.where((overlay==[255, 255, 255, 0]).all(axis=2))] = np.array([0, 0, 0, 0])
    gray_num = random.randint(10, 250)
    scale = random.randint(1, 10) / 10
    x = random.randint(int(xmax*0.1), int(xmax*0.9))
    y = random.randint(int(ymax*0.1), int(ymax*0.9))
    print(x, y)
    overlay[np.where((overlay!=[0,0, 0, 0]).any(axis=2))] = np.array([gray_num, gray_num, gray_num, 1])
    background = logoOverlay(background,overlay,scale=scale,y=y,x=x)
  return background